In [3]:
list_a = [1,2,3,4,5]
list_b = [6,7,8,9,10]


list_c = [x + y for x, y in zip(list_a, list_b)]
dict_c = {x: y for x, y in zip(list_a, list_b)}

In [4]:
list_c

[7, 9, 11, 13, 15]

In [6]:
dict_c

{1: 6, 2: 7, 3: 8, 4: 9, 5: 10}

In [13]:
# On Linux/Mac, you can check:
import subprocess
import os

try:
    # Linux
    result = subprocess.run(['getconf', 'LEVEL1_DCACHE_LINESIZE'], 
                          capture_output=True, text=True)
    if result.returncode == 0:
        print(f"Cache line size: {result.stdout.strip()} bytes")
except:
    print("Cache line size detection not available")

In [14]:
result

CompletedProcess(args=['getconf', 'LEVEL1_DCACHE_LINESIZE'], returncode=64, stdout='', stderr="getconf: no such configuration parameter `LEVEL1_DCACHE_LINESIZE'\n")